In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:


from huggingface_hub import login
import os

token_file_path = 'secrets.txt'

if os.path.exists(token_file_path):
    try:
        with open(token_file_path, 'r') as f:
            hf_token = f.read().strip()  # .strip() removes any leading/trailing whitespace

        login(token=hf_token)
        print("Successfully logged in to Hugging Face!")

    except Exception as e:
        print(f"An error occurred while trying to read the token file or log in: {e}")
else:
    print(f"Token file not found at {token_file_path}. Please create the file and add your token.")



/home/jovyan/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Successfully logged in to Hugging Face!


In [3]:
from src.data_utils import SIFT50MDataset
import os
from datasets import load_dataset, Dataset

sift_dataset = load_dataset(
    'amazon-agi/SIFT-50M',
    #name='closed_ended_comparison',
    name = 'closed_ended_content_level',
    split='train',
    trust_remote_code=True
)
 

# Define the allowed data sources
allowed_values = ["common_voice_de", "common_voice_en", "vctk_en"]

# Define a filter function
def filter_data_source(example):
    return example["data_source"] in allowed_values

filtered_sift_data = sift_dataset.filter(filter_data_source)


In [10]:
len(sift_dataset)

14473775

In [4]:

filtered_sift_data = filtered_sift_data.add_column(
    "data_source_str", filtered_sift_data["data_source"]
)

stratifiable_dataset = filtered_sift_data.class_encode_column("data_source")

train_validation_split = stratifiable_dataset.train_test_split(
    test_size=0.2,
    seed=42,
    stratify_by_column="data_source"
)

train_ds, eval_ds = train_validation_split["train"], train_validation_split["test"]



In [5]:
#train_ds, eval_ds = train_validation_split["train"], train_validation_split["test"]

train_ds = train_ds.remove_columns("data_source").rename_column("data_source_str", "data_source")
eval_ds  = eval_ds.remove_columns("data_source").rename_column("data_source_str", "data_source")


In [6]:
len(eval_ds)

793296

In [9]:


base_datasets_root = "/home/jovyan/.cache/huggingface/datasets"
base_datasets_paths = {
    "common_voice_de": None, # No longer needs a path, handled by load_dataset
    #"multilingual_librispeech_de": None, # No longer needs a path, handled by load_dataset
    "common_voice_en": None, # No longer needs a path, handled by load_dataset
    "vctk_en": "./vctk_corpus" # VCTK still needs a root path for torchaudio
}



In [20]:
sift_iterable_dataset_eval = SIFT50MDataset(sift_dataset=eval_ds, base_datasets_paths=base_datasets_paths)

for x in sift_iterable_dataset_eval:
    print('')
































Path not found for id: common_voice_en_25505298
Skipping entry common_voice_en_25505298, no valid audio found.


























Path not found for id: common_voice_en_27227960
Skipping entry common_voice_en_27227960, no valid audio found.


















Path not found for id: common_voice_en_18971679
Skipping entry common_voice_en_18971679, no valid audio found.
Path not found for id: common_voice_en_19509550
Skipping entry common_voice_en_19509550, no valid audio found.





















Path not found for id: common_voice_en_19869069
Skipping entry common_voice_en_19869069, no valid audio found.


















Path not found for id: common_voice_en_26142243
Skipping entry common_voice_en_26142243, no valid audio found.







Path not found for id: common_voice_en_23887986
Skipping entry common_voice_en_23887986, no valid audio found.


























































Path not found for id: common_voice_en_22316

KeyboardInterrupt: 

In [7]:
%reload_ext autoreload
    
from src.train_qlora import train_model
from transformers import AutoProcessor
from src.data_utils import SIFT50MDataset
from src.data_collator import CustomDataCollator


def main():
    processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct", sampling_rate=16000)
    data_collator = CustomDataCollator(processor)

    sift_iterable_dataset_eval = SIFT50MDataset(sift_dataset=eval_ds, base_datasets_paths=base_datasets_paths)
    sift_iterable_dataset_train = SIFT50MDataset(sift_dataset=train_ds, base_datasets_paths=base_datasets_paths)

    train_model( eval_ds = sift_iterable_dataset_eval,train_ds = sift_iterable_dataset_train, processor=processor, custom_data_collator=data_collator, resume=False)

In [10]:
main()

Loading checkpoint shards: 100%|██████████| 5/5 [00:09<00:00,  1.95s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.38 GiB. GPU 0 has a total capacity of 23.67 GiB of which 2.30 GiB is free. Process 3136679 has 3.69 GiB memory in use. Process 3163281 has 6.62 GiB memory in use. Process 3557306 has 3.44 GiB memory in use. Process 3637560 has 7.61 GiB memory in use. Of the allocated memory 7.02 GiB is allocated by PyTorch, and 367.64 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [2]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [2]:

gc.collect()

88